# Başlatma (Initialization)

"Derin Sinir Ağlarını İyileştirme"nin ilk görevine hoş geldiniz.

Sinir ağınızı eğitmek, ağırlıkların bir başlangıç değerini belirtmeyi gerektirir. İyi seçilmiş bir başlatma yöntemi öğrenmeye yardımcı olacaktır.

Bu çalışma dosyasında, farklı başlatmaların nasıl farklı sonuçlara yol açtığını göreceksiniz.

İyi seçilmiş bir başlatma şunları yapabilir:
- Gradyan inişinin yakınsamasını hızlandırın
- Daha düşük bir eğitim (ve genelleme) hatasına yakınsayan gradyan iniş olasılığını artırın

Başlamak için, paketleri ve sınıflandırmaya çalışacağınız düzlemsel veri kümesini yüklemek için aşağıdaki hücreyi çalıştırın.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import sklearn
import sklearn.datasets

#from init_utils import sigmoid, relu, compute_loss, forward_propagation, backward_propagation
#from init_utils import update_parameters, predict, load_dataset, plot_decision_boundary, predict_dec

%matplotlib inline
plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#@title Yardımcı Fonksiyonlar
def sigmoid(x):
    """
    Compute the sigmoid of x

    Arguments:
    x -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(x)
    """
    s = 1/(1+np.exp(-x))
    return s

def relu(x):
    """
    Compute the relu of x

    Arguments:
    x -- A scalar or numpy array of any size.

    Return:
    s -- relu(x)
    """
    s = np.maximum(0,x)
    
    return s

def forward_propagation(X, parameters):
    """
    Implements the forward propagation (and computes the loss) presented in Figure 2.
    
    Arguments:
    X -- input dataset, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape ()
                    b1 -- bias vector of shape ()
                    W2 -- weight matrix of shape ()
                    b2 -- bias vector of shape ()
                    W3 -- weight matrix of shape ()
                    b3 -- bias vector of shape ()
    
    Returns:
    loss -- the loss function (vanilla logistic loss)
    """
        
    # retrieve parameters
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    z1 = np.dot(W1, X) + b1
    a1 = relu(z1)
    z2 = np.dot(W2, a1) + b2
    a2 = relu(z2)
    z3 = np.dot(W3, a2) + b3
    a3 = sigmoid(z3)
    
    cache = (z1, a1, W1, b1, z2, a2, W2, b2, z3, a3, W3, b3)
    
    return a3, cache

def backward_propagation(X, Y, cache):
    """
    Implement the backward propagation presented in figure 2.
    
    Arguments:
    X -- input dataset, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat)
    cache -- cache output from forward_propagation()
    
    Returns:
    gradients -- A dictionary with the gradients with respect to each parameter, activation and pre-activation variables
    """
    m = X.shape[1]
    (z1, a1, W1, b1, z2, a2, W2, b2, z3, a3, W3, b3) = cache
    
    dz3 = 1./m * (a3 - Y)
    dW3 = np.dot(dz3, a2.T)
    db3 = np.sum(dz3, axis=1, keepdims = True)
    
    da2 = np.dot(W3.T, dz3)
    dz2 = np.multiply(da2, np.int64(a2 > 0))
    dW2 = np.dot(dz2, a1.T)
    db2 = np.sum(dz2, axis=1, keepdims = True)
    
    da1 = np.dot(W2.T, dz2)
    dz1 = np.multiply(da1, np.int64(a1 > 0))
    dW1 = np.dot(dz1, X.T)
    db1 = np.sum(dz1, axis=1, keepdims = True)
    
    gradients = {"dz3": dz3, "dW3": dW3, "db3": db3,
                 "da2": da2, "dz2": dz2, "dW2": dW2, "db2": db2,
                 "da1": da1, "dz1": dz1, "dW1": dW1, "db1": db1}
    
    return gradients

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of n_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters['W' + str(i)] = ... 
                  parameters['b' + str(i)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural networks

    # Update rule for each parameter
    for k in range(L):
        parameters["W" + str(k+1)] = parameters["W" + str(k+1)] - learning_rate * grads["dW" + str(k+1)]
        parameters["b" + str(k+1)] = parameters["b" + str(k+1)] - learning_rate * grads["db" + str(k+1)]
        
    return parameters

def compute_loss(a3, Y):
    
    """
    Implement the loss function
    
    Arguments:
    a3 -- post-activation, output of forward propagation
    Y -- "true" labels vector, same shape as a3
    
    Returns:
    loss - value of the loss function
    """
    
    m = Y.shape[1]
    logprobs = np.multiply(-np.log(a3),Y) + np.multiply(-np.log(1 - a3), 1 - Y)
    loss = 1./m * np.nansum(logprobs)
    
    return loss

def load_cat_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    train_set_x_orig = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
    test_set_x_orig = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T
    
    train_set_x = train_set_x_orig/255
    test_set_x = test_set_x_orig/255

    return train_set_x, train_set_y, test_set_x, test_set_y, classes


def predict(X, y, parameters):
    """
    This function is used to predict the results of a  n-layer neural network.
    
    Arguments:
    X -- data set of examples you would like to label
    parameters -- parameters of the trained model
    
    Returns:
    p -- predictions for the given dataset X
    """
    
    m = X.shape[1]
    p = np.zeros((1,m), dtype = np.int)
    
    # Forward propagation
    a3, caches = forward_propagation(X, parameters)
    
    # convert probas to 0/1 predictions
    for i in range(0, a3.shape[1]):
        if a3[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0

    # print results
    print("Accuracy: "  + str(np.mean((p[0,:] == y[0,:]))))
    
    return p

def plot_decision_boundary(model, X, y):
    # Set min and max values and give it some padding
    x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
    y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole grid
    Z = model(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.ylabel('x2')
    plt.xlabel('x1')
    plt.scatter(X[0, :], X[1, :], c=y, cmap=plt.cm.Spectral)
    plt.show()
    
def predict_dec(parameters, X):
    """
    Used for plotting decision boundary.
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (m, K)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Predict using forward propagation and a classification threshold of 0.5
    a3, cache = forward_propagation(X, parameters)
    predictions = (a3>0.5)
    return predictions

def load_dataset():
    np.random.seed(1)
    train_X, train_Y = sklearn.datasets.make_circles(n_samples=300, noise=.05)
    np.random.seed(2)
    test_X, test_Y = sklearn.datasets.make_circles(n_samples=100, noise=.05)
    # Visualize the data
    plt.scatter(train_X[:, 0], train_X[:, 1], c=train_Y, s=40, cmap=plt.cm.Spectral);
    train_X = train_X.T
    train_Y = train_Y.reshape((1, train_Y.shape[0]))
    test_X = test_X.T
    test_Y = test_Y.reshape((1, test_Y.shape[0]))
    return train_X, train_Y, test_X, test_Y

In [ ]:
# load image dataset: blue/red dots in circles
train_X, train_Y, test_X, test_Y = load_dataset()

Mavi noktaları kırmızı noktalardan ayırmak için bir sınıflandırıcı eğitmek istiyoruz

## 1 - Sinir Ağı modeli

3 katmanlı bir sinir ağı kullanacaksınız (zaten sizin için uygulanmış). Deneyeceğiniz başlatma yöntemleri şunlardır:
- *Sıfır başlatma* -- giriş bağımsız değişkeninde `initialization = "zeros"` ayarı.
- *Rastgele başlatma* -- giriş bağımsız değişkeninde `initialization = "random"` ayarı. Bu, ağırlıkları büyük rastgele değerlere başlatır.
- *He başlatma* -- giriş bağımsız değişkeninde `initialization = "he"` ayarı. Bu, ağırlıkları He ve diğerleri, 2015 tarafından hazırlanan bir makaleye göre ölçeklenen rastgele değerlere başlatır.

**Talimatlar**: Lütfen aşağıdaki kodu hızlıca okuyun ve çalıştırın. Sonraki bölümde, bu `model()`in çağırdığı üç başlatma yöntemini uygulayacaksınız.

In [ ]:
def model(X, Y, learning_rate=0.01, num_iterations=15000, print_cost=True, initialization="he"):
    """
    Implements a three-layer neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (2, number of examples)
    Y -- true "label" vector (containing 0 for red dots; 1 for blue dots), of shape (1, number of examples)
    learning_rate -- learning rate for gradient descent 
    num_iterations -- number of iterations to run gradient descent
    print_cost -- if True, print the cost every 1000 iterations
    initialization -- flag to choose which initialization to use ("zeros","random" or "he")
    
    Returns:
    parameters -- parameters learnt by the model
    """
        
    grads = {}
    costs = [] # to keep track of the loss
    m = X.shape[1] # number of examples
    layers_dims = [X.shape[0], 10, 5, 1]
    
    # Initialize parameters dictionary.
    if initialization == "zeros":
        parameters = initialize_parameters_zeros(layers_dims)
    elif initialization == "random":
        parameters = initialize_parameters_random(layers_dims)
    elif initialization == "he":
        parameters = initialize_parameters_he(layers_dims)

    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # Forward propagation: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID.
        a3, cache = forward_propagation(X, parameters)
        
        # Loss
        cost = compute_loss(a3, Y)

        # Backward propagation.
        grads = backward_propagation(X, Y, cache)
        
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Print the loss every 1000 iterations
        if print_cost and i % 1000 == 0:
            print("Cost after iteration {}: {}".format(i, cost))
            costs.append(cost)
            
    # plot the loss
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

## 2 - Sıfır başlatma (Zero initialization)

Bir sinir ağında başlatılacak iki tür parametre vardır:
- ağırlık (weight) matrisleri $(W^{[1]}, W^{[2]}, W^{[3]}, ..., W^{[L-1]}, W^{[L]})$
- önyargı (bias) vektörleri $(b^{[1]}, b^{[2]}, b^{[3]}, ..., b^{[L-1]}, b^{[L]})$

**Alıştırma**: Tüm parametreleri sıfır olarak başlatmak için aşağıdaki fonksiyonu uygulayın. Daha sonra bunun "simetriyi kırmayı" başaramadığı için iyi çalışmadığını göreceksiniz, ama yine de deneyelim ve ne olacağını görelim. np.zeros((..,..)) doğru şekillerle kullanın.

In [ ]:
# GRADED FUNCTION: initialize_parameters_zeros 

def initialize_parameters_zeros(layers_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the size of each layer.
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    W1 -- weight matrix of shape (layers_dims[1], layers_dims[0])
                    b1 -- bias vector of shape (layers_dims[1], 1)
                    ...
                    WL -- weight matrix of shape (layers_dims[L], layers_dims[L-1])
                    bL -- bias vector of shape (layers_dims[L], 1)
    """
    
    parameters = {}
    L = len(layers_dims)            # number of layers in the network
    
    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = np.zeros((layers_dims[l], layers_dims[l - 1]))
        parameters['b' + str(l)] = np.zeros((layers_dims[l], 1))
        ### END CODE HERE ###
    return parameters

In [ ]:
parameters = initialize_parameters_zeros([3,2,1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

Sıfır başlatmayı kullanarak modelinizi 15.000 iterasyonda eğitmek için aşağıdaki kodu çalıştırın.

In [ ]:
parameters = model(train_X, train_Y, initialization = "zeros")
print ("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)


In [ ]:
print ("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

Performans gerçekten kötü ve maliyet gerçekten düşmüyor ve algoritma rastgele tahminden daha iyi performans göstermiyor. Peki neden? Tahminlerin detaylarına ve karar sınırına bakalım:

In [ ]:
print("predictions_train = " + str(predictions_train))
print("predictions_test = " + str(predictions_test))

In [ ]:
plt.title("Model with Zeros initialization")
axes = plt.gca()
axes.set_xlim([-1.5, 1.5])
axes.set_ylim([-1.5, 1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

Model her örnek için 0 öngörüyor.

Genel olarak, tüm ağırlıkların sıfır olması, ağın simetriyi kıramamasına neden olur. Bu, her katmandaki her nöronun aynı şeyi öğreneceği anlamına gelir ve her katman için $n^{[l]}=1$ ile bir sinir ağını eğitiyor olabilirsiniz ve ağ, lojistik regresyon gibi doğrusal bir sınıflandırıcıdan daha güçlü değildir.

<font color='blue'>
**Hatırlamanız gerekenler**:

- Simetriyi kırmak için $W^{[l]}$ ağırlıkları rastgele başlatılmalıdır.
- Bununla birlikte, $b^{[l]}$ önyargılarını sıfır olarak başlatmakta bir sakınca yoktur. $W^{[l]}$ rastgele başlatıldığı sürece simetri hala bozuktur.


## 3 - Rastgele başlatma

Simetriyi kırmak için ağırlıkları rastgele başlatalım. Rastgele başlatmanın ardından, her nöron girdilerinin farklı bir işlevini öğrenmeye devam edebilir. Bu çalışma dosyasında, ağırlıklar rastgele fakat çok büyük değerlere başlatılırsa ne olacağını göreceksiniz.

**Alıştırma**: Ağırlıklarınızı büyük rastgele değerlere (\*10 ile ölçeklendirilmiş) ve önyargılarınızı sıfıra başlatmak için aşağıdaki fonksiyonları uygulayın. Ağırlıklar için `np.random.randn(..,..) * 10` ve sapmalar için `np.zeros((.., ..))` kullanın. "Rastgele" ağırlıklarınızın bizimkilerle eşleştiğinden emin olmak için sabit bir `np.random.seed(..)` kullanıyoruz, bu nedenle, kodunuzu birkaç kez çalıştırmanız size parametreler için her zaman aynı başlangıç değerlerini veriyorsa endişelenmeyin.

In [ ]:
# GRADED FUNCTION: initialize_parameters_random

def initialize_parameters_random(layers_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the size of each layer.
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    W1 -- weight matrix of shape (layers_dims[1], layers_dims[0])
                    b1 -- bias vector of shape (layers_dims[1], 1)
                    ...
                    WL -- weight matrix of shape (layers_dims[L], layers_dims[L-1])
                    bL -- bias vector of shape (layers_dims[L], 1)
    """
    
    np.random.seed(3)               # This seed makes sure your "random" numbers will be the as ours
    parameters = {}
    L = len(layers_dims)            # integer representing the number of layers
    
    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = np.random.randn(layers_dims[l], layers_dims[l - 1]) * 10
        parameters['b' + str(l)] = np.zeros((layers_dims[l], 1))
        ### END CODE HERE ###

    return parameters

In [ ]:
parameters = initialize_parameters_random([3, 2, 1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

Modelinizi rastgele başlatma kullanarak 15.000 iterastonda eğitmek için aşağıdaki kodu çalıştırın.

In [ ]:
parameters = model(train_X, train_Y, initialization = "random")
print("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)

In [ ]:
print("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

İlk iterasyondan sonra maliyet olarak "inf" görüyorsanız, bunun nedeni sayısal yuvarlamadır; sayısal olarak daha karmaşık bir uygulama bunu düzeltebilir. Ama bu bizim amaçlarımız için endişelenmeye değmez.

Her neyse, simetriyi bozmuşsunuz gibi görünüyor ve bu daha eskisinden daha iyi sonuçlar veriyor. Model artık tüm 0'ları vermiyor.

In [ ]:
print(predictions_train)
print(predictions_test)

In [ ]:
plt.title("Model with large random initialization")
axes = plt.gca()
axes.set_xlim([-1.5, 1.5])
axes.set_ylim([-1.5, 1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

**Gözlemler**:
- Maliyet çok yüksek başlar. Bunun nedeni, büyük rastgele değerli ağırlıklarla, son aktivasyonun (sigmoid) bazı örnekler için 0 veya 1'e çok yakın sonuçlar vermesi ve bu örneği yanlış yaptığında o örnek için çok yüksek bir kayba maruz kalmasıdır. Gerçekten de, $\log(a^{[3]}) = \log(0)$ olduğunda, kayıp sonsuza gider.
- Zayıf başlatma, aynı zamanda optimizasyon algoritmasını yavaşlatan kaybolan/patlayan gradyanlara yol açabilir.
- Bu ağı daha uzun süre eğitirseniz daha iyi sonuçlar görürsünüz, ancak aşırı büyük rastgele sayılarla başlatma optimizasyonu yavaşlatır.

<font color='blue'>
**Özetle**:

- Ağırlıkları çok büyük rastgele değerlere başlatmak iyi çalışmıyor.
- Umarım küçük rastgele değerlerle başlatma daha iyi sonuç verir. Önemli soru şudur: Bu rastgele değerler ne kadar küçük olmalıdır? Bir sonraki bölümde öğrenelim!

## 4 - He Başlatma (He Initialization)

Son olarak, "He Initialization" deneyin; bu, He et al., 2015'in ilk yazarı için adlandırılmıştır. ("Xavier başlatma" diye bir şey duyduysanız, Xavier başlatmanın $W^{[l]}$ ağırlıkları için bir ölçekleme faktörü kullanması dışında bu benzerdir. `sqrt(1./layers_dims[l-1])` burada başlatma işlemi `sqrt(2./layers_dims[l-1])` kullanır.)

**Alıştırma**: Parametrelerinizi He Initialization ile başlatmak için aşağıdaki işlevi uygulayın.

**İpucu**: Bu işlev, önceki `initialize_parameters_random(...)` işlevine benzer. Tek fark, `np.random.randn(..,..)` öğesini 10 ile çarpmak yerine, onu $\sqrt{\frac{2}{\text{önceki katmanın boyutu}} ile çarpmanızdır. }$, ReLU aktivasyonu olan katmanlar için başlatmanın önerdiği şeydir.

In [ ]:
# GRADED FUNCTION: initialize_parameters_he

def initialize_parameters_he(layers_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the size of each layer.
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    W1 -- weight matrix of shape (layers_dims[1], layers_dims[0])
                    b1 -- bias vector of shape (layers_dims[1], 1)
                    ...
                    WL -- weight matrix of shape (layers_dims[L], layers_dims[L-1])
                    bL -- bias vector of shape (layers_dims[L], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layers_dims) - 1 # integer representing the number of layers
     
    for l in range(1, L + 1):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = np.random.randn(layers_dims[l], layers_dims[l - 1]) * np.sqrt(2 / layers_dims[l - 1])
        parameters['b' + str(l)] = np.zeros((layers_dims[l], 1))
        ### END CODE HERE ###
        
    return parameters

In [ ]:
parameters = initialize_parameters_he([2, 4, 1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

Modelinizi He Initialization kullanarak 15.000 iterasyon eğitmek için aşağıdaki kodu çalıştırın.

In [ ]:
parameters = model(train_X, train_Y, initialization = "he")
print("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)

In [ ]:
print("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

In [ ]:
plt.title("Model with He initialization")
axes = plt.gca()
axes.set_xlim([-1.5, 1.5])
axes.set_ylim([-1.5, 1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

**Gözlemler**:
- He initialization'lı model, az sayıda iterasyonda mavi ve kırmızı noktaları çok iyi ayırır.

## 5. Sonuçlar

Üç farklı başlatma türü gördünüz. Aynı sayıda yineleme ve aynı hiperparametreler için karşılaştırma şöyledir:

<table> 
    <tr>
        <td>
        Model
        </td>
        <td>
        Train accuracy
        </td>
        <td>
        Problem/Comment
        </td>
    </tr>
        <td>
        3-layer NN with zeros initialization
        </td>
        <td>
        50%
        </td>
        <td>
        fails to break symmetry
        </td>
    <tr>
        <td>
        3-layer NN with large random initialization
        </td>
        <td>
        83%
        </td>
        <td>
        too large weights 
        </td>
    </tr>
    <tr>
        <td>
        3-layer NN with He initialization
        </td>
        <td>
        99%
        </td>
        <td>
        recommended method
        </td>
    </tr>
</table> 

<font color='blue'>
**Bu çalışma dosyasında hatırlamanız gerekenler**:

- Farklı başlatmalar farklı sonuçlara yol açar
- Simetriyi kırmak ve farklı gizli birimlerin farklı şeyler öğrenebilmesini sağlamak için rastgele başlatma kullanılır
- Çok büyük değerlere başlatmayın
- Başlatma, ReLU aktivasyonlarına sahip ağlar için iyi çalışıyor.